installing required packages

In [ ]:
!pip install -q transformers torch sentence-transformers faiss-cpu flask pyngrok accelerate bitsandbytes

print("all packages installed successfully!")

importing librarires

In [ ]:
import torch
import json
import yaml
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import time

print("Libraries imported successfully!")

Knowledge Base

In [ ]:
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Document 1: Shoplite User Registration Process",
        "content": """To create a Shoplite account, users visit the registration page and provide their email address, password, and basic profile information. Email verification is required within 24 hours of registration. Users can choose between buyer accounts (free) or seller accounts (requires business verification). Creating a Shoplite account is a straightforward process designed to get new users shopping quickly while maintaining security standards. Users begin by visiting the registration page at shoplite.com/register, where they'll need to provide their email address, create a secure password, and enter basic profile information including their full name and preferred display name. During registration, users can choose between two account types: Buyer accounts are completely free and provide immediate access to shopping features, while Seller accounts require additional business verification steps. All new accounts must verify their email address within 24 hours of registration by clicking a verification link sent to their provided email. The registration form includes optional fields for phone number, birth date, and marketing preferences. Users can enable two-factor authentication during signup for enhanced security, though this can also be added later through account settings. Shoplite supports social login options through Google, Facebook, and Apple for faster registration. Once email verification is complete, new users receive a welcome package including a $10 discount code for their first purchase, access to the Shoplite mobile app, and a personalized onboarding tour highlighting key platform features. Account creation typically takes less than 5 minutes, and users can immediately begin browsing products, adding items to wishlists, and following their favorite sellers."""
    },
    {
        "id": "doc2",
        "title": "Document 2: Shoplite Shopping Cart Features",
        "content": """The Shoplite shopping cart allows users to add multiple items from different sellers, apply promotional codes, and save items for later purchase. Cart contents are preserved for logged-in users across sessions. The Shoplite shopping cart system allows users to collect items from multiple sellers in a single, unified cart experience. Items remain in the cart for 30 days for logged-in users, or until the browser session ends for guest users. The cart automatically saves quantity selections, product variations like size and color, and any applicable promotional codes. During the shopping experience, users can adjust quantities, remove items, or save items to their wishlist directly from the cart page. The cart displays real-time pricing updates, including individual item costs, shipping fees from each seller, applicable taxes, and any discount savings. Shoplite's smart cart feature suggests related products and alerts users to limited-time offers on items in their cart. The checkout process begins with shipping address selection, where users can choose from saved addresses or enter a new one. Payment method selection follows, with options including credit cards, PayPal, Apple Pay, Google Pay, and Shoplite's installment payment plans. Users can split payments across multiple methods if needed. Order review displays a detailed breakdown of items by seller, individual shipping costs, estimated delivery dates, and final totals. Users can apply promotional codes at this stage and select gift wrapping options for individual items. The final step requires confirmation of the order, after which users receive an immediate email confirmation and can track their order status through their account dashboard."""
    },
    {
        "id": "doc3",
        "title": "Document 3: Shoplite Payment Methods and Security",
        "content": """Shoplite accepts multiple payment methods including credit cards, debit cards, PayPal, Apple Pay, Google Pay, and installment plans through Klarna and Affirm. Payment security is maintained through SSL encryption and PCI DSS compliance with tokenized credit card storage. Shoplite accepts multiple payment methods to accommodate diverse customer preferences and international shopping needs. Primary payment options include all major credit cards (Visa, Mastercard, American Express, Discover), debit cards with Visa or Mastercard logos, and digital wallets including PayPal, Apple Pay, Google Pay, and Samsung Pay. For larger purchases, Shoplite offers installment payment plans through partnerships with Klarna and Affirm. Payment security is maintained through industry-standard SSL encryption and PCI DSS compliance. All credit card information is tokenized and never stored on Shoplite servers. The platform uses 3D Secure authentication for international transactions and monitors all payments for fraudulent activity using advanced machine learning algorithms. Shoplite Wallet is an optional digital wallet service that allows users to store funds for faster checkout. Users can add money to their wallet through bank transfers, credit cards, or by receiving refunds from returns. Wallet funds never expire and can be used for any purchase on the platform. International customers can pay in their local currency, with automatic conversion handled at competitive exchange rates. Supported currencies include USD, EUR, GBP, CAD, AUD, JPY, and 15 additional regional currencies. For business customers, Shoplite offers corporate payment solutions including purchase orders, net payment terms for qualified accounts, and bulk payment processing for large orders."""
    },
    {
        "id": "doc4",
        "title": "Document 4: Order Tracking and Delivery",
        "content": """Shoplite provides comprehensive order tracking from purchase confirmation through final delivery. Once an order is placed, customers receive an immediate confirmation email with order number and expected processing timeline. Sellers typically process orders within 1-2 business days, after which tracking information becomes available in the customer's account dashboard. The order tracking system displays real-time updates including order confirmation, seller processing, shipment creation, in-transit status, out for delivery, and final delivery confirmation. Customers receive SMS and email notifications at each major milestone, with the option to customize notification preferences in their account settings. Delivery options vary by seller location and product type, but standard options include economy shipping (5-7 business days), standard shipping (3-5 business days), and expedited shipping (1-2 business days). Same-day delivery is available in major metropolitan areas for qualifying local sellers. International shipping is supported with delivery times ranging from 7-21 business days depending on destination. Special delivery services include signature required, adult signature required for age-restricted items, and delivery to pickup locations such as Amazon Lockers or local post offices. Customers can request delivery instructions, specify safe drop-off locations, and schedule delivery windows for an additional fee in supported areas. If delivery issues occur, Shoplite's resolution team works directly with shipping carriers to locate packages and resolve delivery problems. The platform guarantees delivery and provides full refunds for lost shipments."""
    },
    {
        "id": "doc5",
        "title": "Document 5: Return and Refund Policies",
        "content": """Shoplite maintains customer-friendly return policies while balancing seller protection and operational efficiency. Most items can be returned within 30 days of delivery for a full refund, provided they remain in original condition with all packaging, tags, and accessories. The return window extends to 60 days for Shoplite Premium members and during holiday seasons. Return eligibility varies by product category and seller policies. Electronics must be returned in original packaging with all accessories and cannot show signs of use beyond initial inspection. Clothing items must have original tags attached and show no signs of wear, washing, or alterations. Personal care items, food products, and custom-made items are generally non-returnable for health and safety reasons. The return process begins through the customer's account dashboard, where they can initiate return requests, print prepaid return labels, and track return shipment status. Shoplite provides free return shipping for defective items, wrong items shipped, or seller errors. Customer-initiated returns may incur return shipping fees depending on the seller's policy and return reason. Refunds are processed within 3-5 business days of return receipt and inspection. Refunds return to the original payment method for credit card purchases, while PayPal and digital wallet refunds are typically instant. Store credit options are available for customers who prefer immediate purchasing power, often with a 5% bonus credit incentive. Partial refunds may apply for items returned outside the standard return window, items with minor damage, or items missing original packaging."""
    },
    {
        "id": "doc6",
        "title": "Document 6: Product Search and Filtering",
        "content": """Shoplite's advanced search engine helps customers quickly find products among millions of listings from thousands of sellers. The search function supports natural language queries, specific product model numbers, and even image-based searches where customers can upload photos to find similar items. Auto-complete suggestions appear as users type, learning from popular searches and personal browsing history. Search results can be refined using comprehensive filtering options organized into logical categories. Price filters allow range selection or preset price brackets, while seller filters enable customers to shop from specific merchants, seller ratings, or geographic locations. Product condition filters distinguish between new, like-new, good, and acceptable condition items from various sellers. Category-specific filters provide detailed refinement options tailored to each product type. Electronics filters include brand, screen size, storage capacity, and technical specifications. Clothing filters cover size, color, material, brand, and style categories. Home goods filters include dimensions, materials, room type, and design styles. Advanced filtering options include customer rating minimums, shipping speed preferences, availability status, and promotional offer types. The "Prime-eligible" filter shows items available for expedited shipping, while the "Same-day delivery" filter displays locally available products. Eco-friendly filters highlight sustainable products and sellers with environmental certifications. Search results are sortable by relevance, price (low to high or high to low), customer ratings, newest arrivals, and bestselling items."""
    },
    {
        "id": "doc7",
        "title": "Document 7: Product Reviews and Ratings",
        "content": """Shoplite's review system promotes transparency and helps customers make informed purchasing decisions through verified customer feedback. Only customers who have purchased and received items can leave reviews, ensuring authenticity and reducing fake review manipulation. The review system uses a five-star rating scale with additional detailed criteria for comprehensive evaluation. Customer reviews include overall star ratings plus category-specific ratings for product quality, value for money, shipping speed, and seller communication. Written reviews can include photos and videos to provide visual context for other shoppers. The platform encourages detailed feedback by highlighting helpful reviews and rewarding active reviewers with small account credits. Review authenticity is maintained through multiple verification methods including purchase verification, delivery confirmation, and advanced spam detection algorithms. Suspicious review patterns trigger manual review by Shoplite's content moderation team. Sellers cannot alter or delete customer reviews, though they can respond to feedback professionally to address concerns or provide clarifications. The review aggregation system calculates overall product ratings using weighted averages that consider review recency, reviewer credibility, and verified purchase status. Products with higher review volumes and consistent ratings receive priority in search results. Detailed review breakdowns show the distribution of star ratings and highlight common themes in customer feedback. Review guidelines prohibit personal attacks, inappropriate content, competitor manipulation, and reviews unrelated to the product or selling experience."""
    },
    {
        "id": "doc8",
        "title": "Document 8: Seller Account Setup and Management",
        "content": """Becoming a Shoplite seller requires completing a comprehensive verification process designed to ensure marketplace quality and customer protection. Prospective sellers begin by creating a seller account through the seller portal, providing business information including legal business name, tax identification numbers, and primary business address. Individual sellers must provide personal identification, while businesses must submit incorporation documents. The verification process includes identity confirmation through government-issued ID verification, business license validation where applicable, and bank account verification for payment processing. International sellers must provide additional documentation including proof of business registration in their home country and compliance with local tax obligations. Verification typically takes 2-3 business days for domestic sellers and up to 7 business days for international applications. Once approved, sellers access the comprehensive Seller Dashboard for managing their Shoplite presence. The dashboard provides inventory management tools, order processing workflows, customer communication systems, and detailed analytics reporting. Sellers can create product listings with multiple photos, detailed descriptions, pricing strategies, and inventory tracking across multiple fulfillment centers. Seller performance metrics are continuously monitored including order defect rate, late shipment rate, customer service response times, and overall customer satisfaction scores. Maintaining high performance standards is required for continued platform access and affects search result visibility. Top-performing sellers earn "Shoplite Preferred Seller" status with enhanced visibility and marketing support."""
    },
    {
        "id": "doc9",
        "title": "Document 9: Mobile App Features",
        "content": """The Shoplite mobile app provides full platform functionality optimized for smartphone and tablet usage across iOS and Android devices. The app maintains feature parity with the desktop website while adding mobile-specific capabilities including barcode scanning, location-based shopping, push notifications, and mobile payment integration with device wallets. Product discovery through the mobile app includes personalized recommendations based on browsing history, purchase patterns, and location data. The "Nearby Deals" feature highlights products from local sellers with same-day delivery options. Camera search functionality allows users to photograph items and find similar products instantly, perfect for price comparison or finding specific items seen elsewhere. Shopping cart synchronization ensures seamless transitions between mobile and desktop experiences. Users can add items on mobile and complete purchases on desktop, or vice versa. The app supports offline browsing of previously viewed items and wishlists, with automatic synchronization when connectivity returns. Mobile-exclusive deals and flash sales provide additional incentives for app usage. Seller tools within the mobile app enable on-the-go business management including order notifications, customer message responses, and inventory updates. Sellers can photograph and list new items directly from their mobile device, respond to customer inquiries instantly, and track sales performance through mobile analytics dashboards. Security features include biometric authentication through fingerprint or face recognition, secure mobile payment processing, and automatic logout for enhanced account protection."""
    },
    {
        "id": "doc10",
        "title": "Document 10: Customer Support Procedures",
        "content": """Shoplite provides comprehensive customer support through multiple channels designed to resolve issues quickly and efficiently. The primary support system operates 24/7 with average response times under 2 hours for urgent issues and within 24 hours for standard inquiries. Support is available in multiple languages including English, Spanish, French, German, and Japanese. Self-service options through the Help Center include detailed FAQ sections, video tutorials, and step-by-step troubleshooting guides covering common issues like account problems, order tracking, returns, and payment questions. The searchable knowledge base contains over 500 articles regularly updated to address new features and common customer concerns. Live chat support is available directly through the website and mobile app, connecting customers with trained support specialists who can access account information and resolve most issues in real-time. Chat sessions are automatically saved to customer accounts for future reference. For complex technical issues, chat agents can escalate to specialist teams or schedule follow-up calls. Phone support operates with toll-free numbers for domestic customers and international callback options for global users. Phone support hours vary by region but generally provide coverage during local business hours plus extended evening hours. Phone agents have access to comprehensive customer data and can perform account changes, process refunds, and coordinate with sellers for order resolution. Email support handles non-urgent inquiries and provides detailed written responses for complex issues requiring documentation."""
    }
]

print(f"Loaded {len(KNOWLEDGE_BASE)} documents into knowledge base")


loading prompts

In [ ]:

YAML_CONTENT = """
version: "2.0"
created: "2025-09-29"
author: "Patrick Soueid"

base_retrieval_prompt:
  role: "Disciplined Shoplite customer service assistant for accurate, auditable responses."

  goal: "Produce factual answers using only provided documentation or strict refusal when information is unavailable."

  context: "Audience is Shoplite customers seeking accurate information about platform features, policies, and procedures."

  guidelines:
    output:
      format: "Return structured response with Answer and Sources sections only."
      success:
        required_keys: ["answer", "sources", "confidence_level"]
        answer: "Direct response using only provided document information."
        sources: "Specific document titles referenced, formatted as list."
        confidence_level: "High/Medium/Low based on document coverage of question."
      refusal:
        allowed_keys: ["refusal", "suggested_action"]
        answer_key: "MUST NOT be present in refusal output."
        refusal: "1-2 sentence reason why information unavailable in documentation."
        suggested_action: "Alternative way to get accurate information."

  prohibitions:
    - "Information not present in provided documents"
    - "Speculation or assumptions beyond documented facts"
    - "Personal opinions or recommendations not backed by documentation"
    - "Outdated information contradicted by current documents"
    - "Technical details not specified in company documentation"

  ambiguity_handling:
    - "If question scope is unclear, request specific clarification."
    - "If multiple interpretations possible, address most likely interpretation and note alternatives."
    - "Assume standard Shoplite processes unless customer specifies exceptions."

  style:
    deterministic: true
    extra_prose: minimal
    tone: "professional, helpful, concise"
    citations: required

multi_document_synthesis_prompt:
  role: "Expert Shoplite assistant specialized in connecting information across platform documentation."

  goal: "Synthesize information from multiple documents to provide comprehensive answers or refuse when synthesis isn't possible."

  context: "Complex customer inquiries requiring understanding of how different Shoplite systems interact."

  guidelines:
    output:
      format: "Return structured response with synthesis approach and clear source attribution."
      success:
        required_keys: ["answer", "key_points", "sources", "synthesis_quality"]
        answer: "Comprehensive response connecting relevant information from multiple documents."
        key_points: "2-3 most important takeaways as bullet points."
        sources: "All documents used, with specific sections when applicable."
        synthesis_quality: "Assessment of how well documents address the complete question."
      partial_answer:
        allowed_keys: ["partial_answer", "missing_information", "sources", "next_steps"]
        answer_key: "Use partial_answer instead of answer for incomplete coverage."
        missing_information: "Specific gaps in available documentation."
        next_steps: "How customer can get complete information."

  prohibitions:
    - "Connecting information from documents in ways not explicitly supported"
    - "Filling gaps with assumed information not in documentation"
    - "Creating workflows not documented in provided materials"
    - "Contradicting any information present in source documents"

  synthesis_rules:
    - "Only connect information when documents explicitly relate to same processes"
    - "Maintain consistency with all referenced documents"
    - "Flag any apparent contradictions between documents"
    - "Prioritize most recent or specific information when conflicts exist"

  style:
    deterministic: true
    logical_flow: required
    contradiction_handling: explicit

no_context_refusal_prompt:
  role: "Responsible Shoplite assistant that maintains strict information boundaries."

  goal: "Refuse to answer when provided documentation lacks relevant information, with helpful alternatives."

  context: "Customer questions that cannot be answered using available Shoplite documentation."

  guidelines:
    output:
      format: "Return refusal with specific alternative actions only."
      refusal_structure:
        required_keys: ["refusal", "reason", "suggested_actions"]
        answer_key: "MUST NOT be present in refusal responses."
        refusal: "Clear statement that information is not available in current documentation."
        reason: "Specific explanation of what information is missing."
        suggested_actions: "Concrete steps customer can take to get accurate information."

  refusal_categories:
    policy_gaps: "Information not covered in available policy documents."
    technical_details: "Technical specifications not present in documentation."
    real_time_data: "Current status information requiring live system access."
    account_specific: "Personal account details requiring authenticated access."
    edge_cases: "Unusual scenarios not addressed in standard documentation."

  prohibitions:
    - "Guessing or inferring information not explicitly documented"
    - "Providing general advice without documentary support"
    - "Referring to external resources not part of Shoplite documentation"
    - "Making assumptions about undocumented policies or procedures"

  alternative_actions:
    contact_support: "24/7 customer support via live chat, phone, or email"
    help_center: "Self-service resources in Shoplite Help Center"
    account_dashboard: "Personal account settings and order history"
    seller_portal: "Seller-specific resources and support channels"

  style:
    deterministic: true
    empathy: required
    actionable_guidance: required

clarification_request_prompt:
  role: "Precision-focused Shoplite assistant ensuring accurate question interpretation."

  goal: "Request specific clarification for ambiguous questions to prevent incorrect responses."

  context: "Customer questions with multiple possible interpretations or missing critical details."

  guidelines:
    output:
      format: "Return clarification request with specific options when possible."
      clarification_structure:
        required_keys: ["clarification_needed", "possible_interpretations", "specific_question"]
        answer_key: "MUST NOT be present until clarification received."
        clarification_needed: "Explanation of what makes the question ambiguous."
        possible_interpretations: "2-3 specific ways the question could be interpreted."
        specific_question: "Direct question to resolve the ambiguity."
      partial_assistance:
        allowed_keys: ["partial_answer", "clarification_needed", "specific_question"]
        partial_answer: "Any unambiguous parts that can be answered immediately."

  ambiguity_triggers:
    - "Vague timeframes ('recently', 'a while ago', 'soon')"
    - "Generic terms ('account issues', 'payment problems', 'not working')"
    - "Multiple possible subjects ('my order' when customer has multiple orders)"
    - "Unclear process stage ('during checkout' without specifying which step)"
    - "Ambiguous product categories ('electronics' spanning multiple types)"

  clarification_techniques:
    option_based: "Present 2-3 specific options for customer to choose from."
    example_driven: "Use concrete examples to illustrate different interpretations."
    step_specific: "Break down complex processes into specific steps for targeting."
    timeframe_specific: "Request specific dates or timeframes when relevant."

  prohibitions:
    - "Answering the most likely interpretation without confirming"
    - "Providing multiple contradictory answers for different interpretations"
    - "Making assumptions about customer intent without verification"

  style:
    deterministic: true
    option_clarity: required
    customer_friendly: true

escalation_prompt:
  role: "Triage-specialized Shoplite assistant identifying cases requiring human intervention."

  goal: "Recognize escalation scenarios and provide appropriate handoff with immediate available assistance."

  context: "Customer issues exceeding documentation-based support or requiring specialized human expertise."

  guidelines:
    output:
      format: "Return escalation notice with immediate help and clear next steps."
      escalation_structure:
        required_keys: ["immediate_help", "escalation_reason", "specialist_contact", "expectations"]
        answer_key: "Use immediate_help for any documentation-based assistance possible."
        immediate_help: "Any relevant information from documentation that helps while awaiting specialist."
        escalation_reason: "Specific reason why specialist attention is required."
        specialist_contact: "Exact contact method for appropriate specialist team."
        expectations: "Realistic timeline and process for specialist response."

  escalation_triggers:
    security_issues: "Account security, fraud, or unauthorized access concerns"
    payment_disputes: "Billing errors, payment failures, or refund complications"
    technical_failures: "Platform malfunctions or system errors"
    policy_violations: "Seller misconduct or terms of service violations"
    legal_matters: "Compliance, legal disputes, or regulatory questions"
    complex_orders: "Multi-seller, international, or high-value order problems"

  specialist_routing:
    security_team: "Account security and fraud prevention specialists"
    billing_team: "Payment processing and billing dispute specialists"
    technical_team: "Platform engineering and system issue specialists"
    compliance_team: "Legal and regulatory compliance specialists"
    seller_relations: "Seller performance and policy enforcement specialists"

  prohibitions:
    - "Attempting to resolve issues requiring system access or account changes"
    - "Providing legal advice or policy interpretations beyond documentation"
    - "Making commitments about resolution timelines or outcomes"
    - "Handling sensitive security or financial information"

  style:
    deterministic: true
    urgency_appropriate: true
    expectation_management: required

system_instruction: |
  You are a helpful Shoplite customer service assistant. Your role is to provide accurate answers using ONLY the provided documentation context.

  CRITICAL RULES:
  1. Use ONLY information from the provided document snippets
  2. If the context doesn't contain relevant information, say "I don't have information about that in the available documentation"
  3. Cite specific documents when possible
  4. Never make up or assume information
  5. Be concise and helpful

  Context documents will be provided below. Answer the question based only on this context.

prompt_selection_rules:
  context_available: "base_retrieval_prompt"
  multi_document_needed: "multi_document_synthesis_prompt"
  no_relevant_context: "no_context_refusal_prompt"
  question_unclear: "clarification_request_prompt"
  requires_specialist: "escalation_prompt"

system_config:
  response_validation:
    required_structure: true
    citation_format: "Document X: [Title]"
    confidence_threshold: 0.7
    max_context_docs: 5

  output_constraints:
    max_response_length: 500
    prohibited_speculation: true
    required_source_attribution: true
    fallback_behavior: "no_context_refusal_prompt"

  quality_controls:
    fact_checking: "against_provided_docs_only"
    consistency_check: "cross_document_validation"
    completeness_threshold: 0.8
    escalation_confidence: 0.9
"""



print("Prompts loaded successfully!")



loading the llm

In [ ]:


print("loading LLM model (this may take a few minutes)...")

model_name = "Qwen/Qwen2-7B-Instruct"

print(f"downloading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

MODEL_DEVICE = next(model.parameters()).device #####testing

print("LLM model loaded successfully!")
print(f"Model device: {model.device}")
if torch.cuda.is_available():
    print(f"Model memory footprint: ~{torch.cuda.memory_allocated() / 1024**3:.2f} GB")

print("Loading embedding model...")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedding model loaded successfully!")


RAG

In [ ]:
print("Building RAG pipeline...")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("cleared GPU cache")

embedding_model = embedding_model.cpu()

print("creating document embeddings...")
documents_text = [doc["content"] for doc in KNOWLEDGE_BASE]
document_embeddings = embedding_model.encode(documents_text, show_progress_bar=True)

print("building FAISS index...")
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings.astype('float32'))
print(f"FAISS index built with {index.ntotal} documents")


def select_prompt_type(query, context_docs, retrieval_scores=None): ## to be fixed later on

    
    if not context_docs or len(context_docs) == 0:
        return "no_context_refusal_prompt"
    
    if retrieval_scores and max(retrieval_scores) < 0.5:
        return "no_context_refusal_prompt"
    
    ambiguous_keywords = ['recently', 'a while ago', 'some', 'issues', 'problems', 'not working']
    if any(keyword in query.lower() for keyword in ambiguous_keywords):
        return "clarification_request_prompt"
    
    escalation_keywords = ['fraud', 'hack', 'stolen', 'unauthorized', 'legal', 'lawsuit', 'scam']
    if any(keyword in query.lower() for keyword in escalation_keywords):
        return "escalation_prompt"
    
    if len(context_docs) >= 3:
        return "multi_document_synthesis_prompt"
    
    return "base_retrieval_prompt"


def retrieve_documents(query, top_k=3):
   
    query_embedding = embedding_model.encode([query])

    distances, indices = index.search(query_embedding.astype('float32'), top_k)

    retrieved_docs = []
    for idx, distance in zip(indices[0], distances[0]):
        doc = KNOWLEDGE_BASE[idx]

        similarity_score = 1 / (1 + distance)
        retrieved_docs.append({
            "title": doc["title"],
            "content": doc["content"][:800],  #limit for promp size
            "similarity": float(similarity_score),
            "doc_id": doc["id"]
        })

    return retrieved_docs

def generate_response(query, context_docs):
    

    context = "\n\n".join([
        f"{doc['title']}:\n{doc['content']}"
        for doc in context_docs
    ])

    # Shorter, more concise system instruction
    system_instruction = """You are a Shoplite customer service assistant. Answer questions using ONLY the provided documentation. Be concise and cite sources."""

    # a small prompt for now in order to fix the prompting issue.
    prompt = f"""{system_instruction}

CONTEXT:
{context}

QUESTION: {query}

ANSWER:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,        
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            num_beams=1,              
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

  
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    
    if "ANSWER:" in full_response:
        response = full_response.split("ANSWER:")[-1].strip()
    else:
      
        response = full_response[len(prompt):].strip()

  
    sentences = response.split('.')
    if len(sentences) > 4:
        response = '. '.join(sentences[:4]) + '.'

    return response

def rag_pipeline(query):
   
    print(f" Processing query: {query}")

    retrieved_docs = retrieve_documents(query, top_k=2)
    print(f" Retrieved {len(retrieved_docs)} documents")
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"   {i}. {doc['title'].split(':')[1].strip() if ':' in doc['title'] else doc['title']} (similarity: {doc['similarity']:.3f})")

  
    print(f" generating response...")
    response = generate_response(query, retrieved_docs)

    sources = [doc["title"] for doc in retrieved_docs]
    top_similarity = retrieved_docs[0]["similarity"]

    if top_similarity > 0.7:
        confidence = "High"
    elif top_similarity > 0.5:
        confidence = "Medium"
    else:
        confidence = "Low"

    result = {
        "answer": response,
        "sources": sources,
        "confidence_level": confidence,
        "top_similarity": float(top_similarity)
    }

    print(f"✅ Response generated (confidence: {confidence})")

    return result


API

In [ ]:

print(" Setting up Flask API...")

from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    
    return jsonify({
        "status": "healthy",
        "model": model_name,
        "documents_loaded": len(KNOWLEDGE_BASE),
        "faiss_index_size": index.ntotal,
        "embedding_model": "all-MiniLM-L6-v2"
    })

@app.route('/chat', methods=['POST'])
def chat():
  
    try:
        # Parse request
        data = request.json
        query = data.get('question', '')

        if not query:
            return jsonify({"error": "No question provided"}), 400

        print(f"\n{'='*60}")
        print(f" Received question: {query}")
        print(f"{'='*60}")

        # Run RAG pipeline
        result = rag_pipeline(query)

        print(f"{'='*60}")
        print(f" Response sent")
        print(f"{'='*60}\n")

        return jsonify(result)

    except Exception as e:
        print(f" Error in /chat endpoint: {str(e)}")
        return jsonify({"error": str(e)}), 500

@app.route('/ping', methods=['POST'])
def ping():
    try:
        data = request.json
        query = data.get('question', '')

        if not query:
            return jsonify({"error": "No question provided"}), 400

        print(f"\n Ping request: {query}")

        simple_prompt = f"Q: {query}\nA:"

        inputs = tokenizer(simple_prompt, return_tensors="pt", truncation=True, max_length=128).to(MODEL_DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=20,     
                min_new_tokens=5,     
                temperature=0.5,      
                do_sample=False,      
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.split("A:")[-1].strip() if "A:" in response else response[len(simple_prompt):].strip()

        response = response.split('.')[0] + '.' if '.' in response else response

        return jsonify({
            "response": response,
            "mode": "direct_llm"
        })

    except Exception as e:
        print(f" Error in /ping endpoint: {str(e)}")
        return jsonify({"error": str(e)}), 500

def run_flask():
  
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)



NGROK Tunnel

In [ ]:

print(" Setting up ngrok tunnel...")
print("=" * 60)
print("IMPORTANT: You need an ngrok authentication token")
print("Get your free token at: https://dashboard.ngrok.com/get-started/your-authtoken")
print("=" * 60)

from pyngrok import ngrok
import time

ngrok.kill()

ngrok_token = input("Enter your ngrok authentication token: ").strip()

if not ngrok_token:
    print(" Error: ngrok token is required!")
    print("Please run this cell again and provide your token.")
else:
    ngrok.set_auth_token(ngrok_token)
    print(" ngrok authentication configured")

    print("Starting Flask server in background...")
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()

    time.sleep(5) 
    print(" Flask server started on port 5000")

    print("Creating ngrok tunnel...")
    public_url = ngrok.connect(5000)

    public_url_str = str(public_url).split('"')[1] if '"' in str(public_url) else str(public_url)

   
    print(f" Public URL: {public_url_str}")
    
   



Testing

In [ ]:

print("\n Running system tests...")
print("=" * 60)

public_url_str = str(public_url).split('"')[1] if '"' in str(public_url) else str(public_url)

test_questions = [
    "How do I create a seller account on Shoplite?",
    "What payment methods are accepted?",
    "How long do items stay in my shopping cart?",
    "What are Shoplite's return policies?",
    "How do customer reviews work?"
]

print("Testing RAG System with Sample Questions")
print("=" * 60)

print("\n Test 1: Health Check")
print("-" * 60)
try:
    import requests
    response = requests.get(f"{public_url_str}/health")
    health_data = response.json()
    print(" Health check passed!")
    print(f"   Status: {health_data.get('status')}")
    print(f"   Model: {health_data.get('model')}")
    print(f"   Documents: {health_data.get('documents_loaded')}")
    print(f"   FAISS Index: {health_data.get('faiss_index_size')}")
except Exception as e:
    print(f" Health check failed: {str(e)}")


print("\n Test 2: RAG-Powered Q&A (/chat endpoint)")
print("-" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 Question {i}: {question}")
    print("-" * 40)
    
    try:
        response = requests.post(
            f"{public_url_str}/chat",
            json={"question": question},
            timeout=120  # Increased to 120 seconds
        )
        
        if response.status_code == 200:
            result = response.json()
            
            print(f" Answer: {result.get('answer', 'N/A')[:200]}...")
            print(f" Sources: {', '.join(result.get('sources', [])[:2])}")
            print(f" Confidence: {result.get('confidence_level', 'N/A')}")
            print(f" Similarity: {result.get('top_similarity', 0):.3f}")
        else:
            print(f" Error: Status code {response.status_code}")
            print(f"   Response: {response.text}")
    
    except Exception as e:
        print(f" Request failed: {str(e)}")
    
    time.sleep(2)



print("\n Test 3: Direct LLM Interaction (/ping endpoint)")
print("-" * 60)

ping_test = "Hello, what can you help me with?"
print(f" Question: {ping_test}")
print("-" * 40)

try:
    response = requests.post(
        f"{public_url_str}/ping",
        json={"question": ping_test},
        timeout=30
    )

    if response.status_code == 200:
        result = response.json()
        print(f" Response: {result.get('response', 'N/A')[:200]}...")
        print(f" Mode: {result.get('mode', 'N/A')}")
    else:
        print(f" Error: Status code {response.status_code}")

except Exception as e:
    print(f" Request failed: {str(e)}")


